In [1]:
import pandas as pd
import numpy as np
import pyod
np.set_printoptions(precision=3)

In [2]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.knn import KNN

In [3]:
trainig_set = pd.read_csv('C:/Users/andre/Desktop/trainig_set.csv',sep=',',header=0)
testing_set = pd.read_csv('C:/Users/andre/Desktop/testing_set.csv',sep=',',header=0)

C:\Users\andre\Anaconda3\envs\security\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
trainig_set

,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,624,192.168.4.118-203.73.24.75-4504-80-6,192.168.4.118,4504,203.73.24.75,80,6,12/06/2010 08:34:32 AM,3974862,29,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
1,625,192.168.4.118-203.73.24.75-4504-80-6,192.168.4.118,4504,203.73.24.75,80,6,12/06/2010 08:34:36 AM,63,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
2,626,192.168.4.118-203.73.24.75-4505-80-6,192.168.4.118,4505,203.73.24.75,80,6,12/06/2010 08:34:36 AM,476078,2,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
3,627,192.168.4.118-203.73.24.75-4505-80-6,192.168.4.118,4505,203.73.24.75,80,6,12/06/2010 08:34:37 AM,151,2,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
4,628,192.168.4.118-203.73.24.75-4506-80-6,192.168.4.118,4506,203.73.24.75,80,6,12/06/2010 08:34:37 AM,472507,2,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
5,629,192.168.4.118-203.73.24.75-4506-80-6,192.168.4.118,4506,203.73.24.75,80,6,12/06/2010 08:34:37 AM,138,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
6,630,192.168.4.118-203.73.24.75-4507-80-6,192.168.4.118,4507,203.73.24.75,80,6,12/06/2010 08:34:37 AM,469596,2,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
7,631,192.168.4.118-203.73.24.75-4507-80-6,192.168.4.118,4507,203.73.24.75,80,6,12/06/2010 08:34:37 AM,1978,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
8,632,192.168.4.118-203.73.24.75-4508-80-6,192.168.4.118,4508,203.73.24.75,80,6,12/06/2010 08:34:37 AM,468629,2,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos
9,633,192.168.4.118-203.73.24.75-4508-80-6,192.168.4.118,4508,203.73.24.75,80,6,12/06/2010 08:34:38 AM,102,1,...,0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,ddos


In [31]:
trainingSet = trainig_set.values
for i in range (20000):
    if(trainingSet[i, 84] == "ddos"):
        trainingSet[i, 84] = 1
    else:
        trainingSet[i, 84] = 0

np.random.shuffle(trainingSet)
y_training = trainingSet[:, 84].astype(int)
trainingSet = np.delete(trainingSet, 84, axis=1)

trainingSet = np.delete(trainingSet, 7, axis=1)
trainingSet = np.delete(trainingSet, 4, axis=1)
trainingSet = np.delete(trainingSet, 2, axis=1)
trainingSet = np.delete(trainingSet, 1, axis=1)
trainingSet = np.delete(trainingSet, 0, axis=1)

#trainingSet = trainingSet.astype(float)

print("TrainingSet infs and Nans: ")
for i in range (0, 20000):
    for j in range(0, 79):
        #if (j != 7 and j != 4 and j != 2 and j != 1 and j != 0):
        trainingSet[i, j] = float(trainingSet[i, j])
        if (np.isinf(trainingSet[i, j]) or np.isnan(trainingSet[i, j])):
            trainingSet[i, j] = 1.7*np.e**308
            #print("(",i,",",j,")",trainingSet[i, j])



testingSet = testing_set.values
for i in range (10**3):
    if(testingSet[i, 84] == "ddos"):
        testingSet[i, 84] = 1
    else:
        testingSet[i, 84] = 0

np.random.shuffle(testingSet)
y_test = testingSet[:, 84].astype(int)
testingSet = np.delete(testingSet, 84, axis=1)

testingSet = np.delete(testingSet, 7, axis=1)
testingSet = np.delete(testingSet, 4, axis=1)
testingSet = np.delete(testingSet, 2, axis=1)
testingSet = np.delete(testingSet, 1, axis=1)
testingSet = np.delete(testingSet, 0, axis=1)

#testingSet = testingSet.astype(float)

print("TestingSet infs and Nans: ")
for i in range (0, 1000):
    for j in range(0, 79):
        #if (j != 7 and j != 4 and j != 2 and j != 1 and j != 0):
        testingSet[i, j] = float(testingSet[i, j])
        if (np.isinf(testingSet[i, j]) or np.isnan(testingSet[i, j])):
            testingSet[i, j] = 1.7*np.e**308
            #print("(",i,",",j,")",testingSet[i, j])

TrainingSet infs and Nans: 
TestingSet infs and Nans: 


In [40]:
clf_name = 'KNN'
clf = KNN(contamination = 0.2)
clf.fit(trainingSet, y_training) 

C:\Users\andre\Anaconda3\envs\security\lib\site-packages\pyod\models\base.py:344: UserWarning: y should not be presented in unsupervised learning.
  "y should not be presented in unsupervised learning.")


KNN(algorithm='auto', contamination=0.2, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [41]:
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

y_test_pred = clf.predict(testingSet)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(testingSet)  # outlier scores

In [43]:
from pyod.utils.data import evaluate_print

# evaluate and print the results
#print("\nOn Training Data:")
#evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")

truePositive = 0
trueNegative = 0
falsePositive = 0
falseNegative = 0

for i in range(y_test.size):
    if(y_test[i] == 1 and y_test_pred[i] == 1):
        truePositive = truePositive+1
    elif (y_test[i] == 0 and y_test_pred[i] == 0):
        trueNegative = trueNegative+1
    elif (y_test[i] == 0 and y_test_pred[i] == 1):
        falsePositive = falsePositive+1
    else:
        falseNegative = falseNegative+1

print(truePositive, " | ", falsePositive)
print(falseNegative, " | ", trueNegative,"\n")

evaluate_print(clf_name, y_test, y_test_scores)


On Test Data:
78  |  165
101  |  656 

KNN ROC:0.7229, precision @ rank n:0.3911


In [38]:
from pyod.utils.example import visualize

visualize(clf_name, trainingSet, y_train, testingSet, y_test, y_train_pred,
          y_test_pred, show_figure=True, save_figure=False)

NameError: name 'y_train' is not defined